In [19]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import time
import requests

In [16]:
songs_graph = {'Do I Wanna Know?': ['indie rock', 'indie', 'alternative'],
 'indie rock': ['Do I Wanna Know?', 'Mr. Brightside', 'R U Mine?'],
 'indie': ['Do I Wanna Know?',
  'Mr. Brightside',
  'R U Mine?',
  'Pumped Up Kicks',
  'The Less I Know the Better'],
 'alternative': ['Do I Wanna Know?',
  'Losing My Religion',
  'Seven Nation Army',
  'The Less I Know the Better'],
 'Mr. Brightside': ['rock', 'indie', 'indie rock'],
 'rock': ['Mr. Brightside',
  'Losing My Religion',
  'Seven Nation Army',
  'Smells Like Teen Spirit'],
 'R U Mine?': ['indie rock', 'indie', 'british'],
 'british': ['R U Mine?'],
 'Take on Me': ['80s', 'pop', 'new wave'],
 '80s': ['Take on Me'],
 'pop': ['Take on Me', 'Sorry'],
 'new wave': ['Take on Me'],
 'Pumped Up Kicks': ['indie', 'indie pop', 'catchy'],
 'indie pop': ['Pumped Up Kicks'],
 'catchy': ['Pumped Up Kicks'],
 'Losing My Religion': ['rock', '90s', 'alternative'],
 '90s': ['Losing My Religion','pop'],
 'Seven Nation Army': ['rock', 'alternative rock', 'alternative'],
 'alternative rock': ['Seven Nation Army'],
 'Sorry': ['pop', 'justin bieber', 'dancehall'],
 'justin bieber': ['Sorry'],
 'dancehall': ['Sorry'],
 'Smells Like Teen Spirit': ['Grunge', 'rock', 'Nirvana'],
 'Grunge': ['Smells Like Teen Spirit'],
 'Nirvana': ['Smells Like Teen Spirit'],
 'The Less I Know the Better': ['indie', 'psychedelic', 'alternative'],
 'psychedelic': ['The Less I Know the Better']}

# _graph = nx.Graph()

# _graph.add_nodes_from(list(songs_graph.keys()))

# for node in list(songs_graph.keys()):
#     for edge in songs_graph[node]:
#         _graph.add_edge(node,edge)

In [56]:
def lastfm_request(api_method,artist='',track=''):
    
    if api_method==0:
        method = f"track.gettoptags&artist={artist}&track={track}"
    else:
        method = f"track.search&track={track}&limit=1"
    
    key ="3dc1e72157f70010ef87db6b769fbfe7"
    API_URL = f'http://ws.audioscrobbler.com/2.0/?method={method}&api_key={key}&format=json'
    return requests.request(method='get', url=API_URL).json()


song_name = input("Diga uma música que você gosta")
result = lastfm_request(1,track=song_name)['results']['trackmatches']['track'][0]
tags = lastfm_request(0,result['artist'],result['name'])['toptags']['tag'][:3]

if result['name'] not in songs_graph:
        songs_graph[result['name']] = []
        for tag in tags:
            songs_graph[result["name"]].append(tag["name"])
            if tag["name"] not in songs_graph:
                songs_graph[tag["name"]] = [result["name"]]
            else:
                songs_graph[tag["name"]].append(result["name"])
                


Diga uma música que você gostaUndisclosed desires


In [93]:
%%time
def dfs_caminhos(grafo, inicio, fim):
    pilha = [(inicio, [inicio])]
    depth = 0
    while pilha:
        vertice, caminho = pilha.pop()
        for proximo in set(grafo[vertice]) - set(caminho):
            if proximo == fim:
                yield caminho + [proximo]
            else:
                pilha.append((proximo, caminho + [proximo]))

all_paths2 = []
best_match = {
    'name': '',
    'lenght': 0,
    'number': 0
}
for node in songs_graph:
    answers = [len(x) for x in list(dfs_caminhos(songs_graph, node, result['name']))]
#     lol = str(answers.count(min(answers)))+' '+str(min(answers)) if answers  else 'None'
#     lel = f'{} {}' if answers else 'None'
    if answers:
        num_paths = answers.count(min(answers))
        path_lenght = min(answers)
        if path_lenght < best_match['lenght'] or best_match['lenght']==0 or (path_lenght ==best_match['lenght'] and num_paths >  best_match['number']) :
            best_match['name'], best_match['lenght'], best_match['number'] = node, path_lenght, num_paths
    
#     all_paths2.append(f'{node}: {lel}')
#     print(node + ": "+str(lol))
print(best_match)

{'name': 'alternative', 'lenght': 2, 'number': 1}
CPU times: user 18.5 ms, sys: 2.88 ms, total: 21.4 ms
Wall time: 18.3 ms


In [92]:
%%time
def _build_paths_from_predecessors(sources, target, pred):

    if target not in pred or {target}==sources:
        return None

    seen = {target}
    stack = [[target, 0]]
    top = 0
    while top >= 0:
        node, i = stack[top]
        if node in sources:
            yield [p for p, n in reversed(stack[: top + 1])]
        if len(pred[node]) > i:
            stack[top][1] = i + 1
            next = pred[node][i]
            if next in seen:
                continue
            else:
                seen.add(next)
            top += 1
            if top == len(stack):
                stack.append([next, 0])
            else:
                stack[top][:] = [next, 0]
        else:
            seen.discard(node)
            top -= 1

def predecessor(G, source):

    if source not in G:
        raise nx.NodeNotFound(f"Source {source} not in G")

    level = 0  # the current level
    nextlevel = [source]  # list of nodes to check at next level
    seen = {source: level}  # level (number of hops) when seen in BFS
    pred = {source: []}  # predecessor dictionary
    while nextlevel:
        level = level + 1
        thislevel = nextlevel
        nextlevel = []
        for v in thislevel:
            for w in G[v]:
                if w not in seen:
                    pred[w] = [v]
                    seen[w] = level
                    nextlevel.append(w)
                elif seen[w] == level:  
                    pred[w].append(v)  

    return pred

# all_paths = []
best_match = {
    'name': '',
    'lenght': 0,
    'number': 0
}
for node in songs_graph:
    parentage = predecessor(songs_graph, node)
    paths = list(_build_paths_from_predecessors({node}, result['name'], parentage))
    if paths != []:
        num_paths = len(paths)
        path_lenght = len(paths[0])
        if path_lenght < best_match['lenght'] or best_match['lenght']==0 or (path_lenght ==best_match['lenght'] and num_paths >  best_match['number']) :
            best_match['name'], best_match['lenght'], best_match['number'] = node, path_lenght, num_paths
    
best_match       
    
    
#     if paths == []:
#         all_paths.append(f'{node}: None')
#     else:
#         num = len(paths[0])
#         all_paths.append(f'{node}: {len(paths)} {num} ')

# print(all_paths)
    
# parentage = predecessor(songs_graph, 'Take on Me')
# list(_build_paths_from_predecessors({'Take on Me'}, 'Undisclosed Desires', parentage))

CPU times: user 1.01 ms, sys: 0 ns, total: 1.01 ms
Wall time: 1.01 ms


{'name': 'alternative', 'lenght': 2, 'number': 1}